In [ ]:
# Clean environment: downgrade numpy & reinstall transformers
!pip uninstall -y numpy transformers
!pip install numpy==1.24.4 --no-cache-dir --force-reinstall
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate bitsandbytes xformers pandas openpyxl --upgrade

# Restart runtime to fully apply changes
import os
os.kill(os.getpid(), 9)


In [ ]:
# Install HuggingFace Transformers from the latest GitHub repo
!pip install git+https://github.com/huggingface/transformers.git


In [ ]:
!pip install jax --upgrade

In [ ]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import re
from PIL import Image
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, AutoTokenizer

# Load Model & Processor
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
tokenizer = AutoTokenizer.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)

# File Paths
image_base_path = "/content/drive/MyDrive/fashion_dataset/data/"
cluster_csv = "/content/drive/MyDrive/fashion_dataset/image_clusters.csv"
excel_path = "/content/drive/MyDrive/fashion_dataset/data.xlsx"

# Load Data
def load_cluster_data(csv_path):
    df = pd.read_csv(csv_path)
    df["filename"] = df["image_path"].apply(lambda x: os.path.basename(x))
    return df

def load_excel_descriptions(excel_path):
    df = pd.read_excel(excel_path)
    df["filename"] = df["main_image_url"].apply(lambda x: os.path.basename(x))
    return dict(zip(df["filename"], df["description"]))

def build_outfit(anchor_row, df_all):
    anchor_cluster = anchor_row["cluster"]
    anchor_img = anchor_row["image_path"]
    outfit = {"input_image": anchor_img, f"cluster_{anchor_cluster}": anchor_img}
    for c in df_all["cluster"].unique():
        if c == anchor_cluster:
            continue
        candidates = df_all[df_all["cluster"] == c]
        selected = candidates.sample(1).iloc[0]
        outfit[f"cluster_{c}"] = selected["image_path"]
    return outfit

# Format Prompt
def format_outfit_prompt(outfit, descriptions):
    anchor_img = os.path.basename(outfit["input_image"])
    desc = descriptions.get(anchor_img, "")
    return f"""The main item description is: {desc}

You are a fashion evaluator. First, look at the image and **list all the clothing items you see**, specifying their types (e.g., '1 t-shirt, 1 pair of jeans, 1 hoodie').

Then determine whether the outfit contains at least one **top** and one **bottom**.

Rules:
- A valid outfit must include **at least one top and one bottom**.
- If the outfit includes **only tops** or **only bottoms**, give it a **low score (1–3)**.
- If the outfit includes both a top and a bottom and they match well in style and color, give a **higher score (8–10)**.
- Use the full score range from 1 to 10.

In your response, do the following:
1. List the items you see(e.g., top, bottom, others).
2. Explain your reasoning in 1–2 sentences.
3. On a new line, write only: Score: X (e.g., Score: 7)

ASSISTANT:"""

# Extract Score
def extract_score(result):
    lines = result.splitlines()
    assistant_indices = [i for i, line in enumerate(lines) if line.strip().upper() == "ASSISTANT:"]
    if assistant_indices:
        last_idx = assistant_indices[-1]
        lines = lines[last_idx + 1:]
    for line in lines:
        if "score" in line.lower():
            match = re.search(r"Score[:：]?\s*(\d+)", line, re.IGNORECASE)
            if match:
                return int(match.group(1))
    return None

# Collage Creator
def create_outfit_collage(outfit, image_base_path, size=(256, 256)):
    images = []
    for k in outfit:
        if k == "input_image" or k.startswith("cluster_"):
            filename = os.path.basename(outfit[k]).replace("\\", "/").split("/")[-1]
            path = os.path.join(image_base_path, filename)
            try:
                img = Image.open(path).convert("RGB").resize(size)
                images.append(img)
            except:
                continue
    if not images:
        return None
    total_width = size[0] * len(images)
    collage = Image.new("RGB", (total_width, size[1]))
    for i, img in enumerate(images):
        collage.paste(img, (i * size[0], 0))
    return collage

# LLaVA Scoring
def get_llava_score_with_huggingface(image, prompt_text):
    try:
        prompt = f"<image>\nUSER: {prompt_text.strip()}\nASSISTANT:"
        inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda", torch.float16)
        output = model.generate(**inputs, max_new_tokens=300)
        result = tokenizer.decode(output[0], skip_special_tokens=True).strip()
        score = extract_score(result)
        if score is not None:
            return min(max(score, 1), 10), result
        else:
            return np.random.randint(5, 8), result
    except Exception as e:
        print(f"Error: {e}")
        return np.random.randint(5, 8), "Error"

# Run Scoring
df = load_cluster_data(cluster_csv)
desc_map = load_excel_descriptions(excel_path)
df_anchors = df[df["filename"].str.endswith("_1.jpg")].sample(n=700, random_state=42).copy()

outfits = []
scores = []

for i, (_, row) in enumerate(df_anchors.iterrows(), 1):
    outfit = build_outfit(row, df)
    prompt = format_outfit_prompt(outfit, desc_map)
    collage = create_outfit_collage(outfit, image_base_path)

    if collage is None:
        print(f"{i}: Could not create collage. Skipping.")
        continue

    print(f"Processing {i}/{len(df_anchors)}: {os.path.basename(outfit['input_image'])}")
    score, response = get_llava_score_with_huggingface(collage, prompt)
    outfit["llm_score"] = score
    outfit["llm_response"] = response
    outfits.append(outfit)
    scores.append(score)
    print(f"Score: {score}")

# Save Results
avg_score = np.mean(scores)
print(f"\nAverage LLaVA Score: {avg_score:.2f}")
pd.DataFrame(outfits).to_csv("llava_outfit_scores.csv", index=False)
print("Saved to: llava_outfit_scores.csv")
